### Simple Maxtrix Factorization

- https://www.analyticsvidhya.com/blog/2018/01/factorization-machines/

The simplest version of maxtri factorization in recommender system (e.g. movie rating) is just

\\[
R \approx PQ^\top,
\\]

where \\(R\\) is the rating matrix with \\(m\\) rows and \\(n\\) columns, each row represents the rating vector for a user to each movie; \\(P\\) has \\(m\\) rows and \\(k\\) columns, each row is a user vector with latent features; \\(Q^\top\\) has \\(k\\) rows and \\(n\\) columns, each column is a movie vector with latent features.

In [1]:
import numpy as np
import pandas as pd
import warnings

In [2]:
data = {'m1': [5., 4., 1., 1., np.nan],
        'm2': [3., np.nan, 1., np.nan, 1.],
        'm3': [np.nan, np.nan, np.nan, np.nan, 5.],
        'm4': [1., 1., 5., 4., 4.]}
data = pd.DataFrame(data)
data

,m1,m2,m3,m4
0,5.0,3.0,NaN,1.0
1,4.0,NaN,NaN,1.0
2,1.0,1.0,NaN,5.0
3,1.0,NaN,NaN,4.0
4,NaN,1.0,5.0,4.0


To make the matrix factorization good, we need to reduce the residuals of approximated matrix and raw matrix:

\\[
\begin{aligned}
p_{ik}, q_{kj} &= \arg\min_{p_{ik}, q_{kj}} \sum_{i, j}\left(r_{ij} - \hat{r_{ij}}\right)^2 \\
               &= \arg\min_{p_{ik}, q_{kj}} \sum_{i, j}\left(r_{ij} - p_{i}^\top q_{j}\right)^2 \\
               &= \arg\min_{p_{ik}, q_{kj}} \sum_{i, j}\left(r_{ij} - \sum_{k=1}^{K}p_{ik}q_{kj}\right)^2,
\end{aligned}
\\]

the direct way to solve this is **gradient descent**, the gradient for \\(p_{ik}\\) and \\(q_{kj}\\) are

\\[
\begin{aligned}
\frac{\partial}{\partial p_{ik}} =& -2\left(r_{ij} - \sum^{K}_{k=1}p_{ik}q_{kj}\right)q_{kj}\\
\frac{\partial}{\partial q_{kj}} =& -2\left(r_{ij} - \sum^{K}_{k=1}p_{ik}q_{kj}\right)p_{ik}
\end{aligned}
\\]

and then apply the update rule. In practice we can also add regularization on it to prevent overfitting.

In [3]:
def matrix_factorization(R, K, lr=1e-3, max_iter=5000, tol=1e-6, random_state=42):
    """ Simple matrix factorization
    Params
    ------
    R           : ndarray, rating matrix (m, n), fill 0.0 for NA value
    K           : int, latent vector length
    lr          : learning rate
    max_iter    : max number of iterations
    tol         : tolerance
    random_state: random seed
    
    Returns
    -------
    P           : ndarray, (m, K)
    Q^T         : ndarray, (K, n)
    """
    
    M, N = R.shape
#     np.random.seed(random_state)
    P = np.random.rand(M, K)
    Q_T = np.random.rand(K, N)
    
    for it in range(max_iter):
        # update for each p_ik, q_kj
        for i in range(M):
            for j in range(N):
                if R[i, j] < 0.0:
                    R[i, j] = 0.0
                err = R[i, j] - P[i, :].dot(Q_T[:, j])
                for k in range(K):
                    P[i, k] += 2 * lr * err * Q_T[k, j] 
                    Q_T[k, j] += 2 * lr * err * P[i, k] 
        
        R_hat = P.dot(Q_T)
        if np.sqrt(np.mean((R - R_hat) ** 2))  < tol:
            break
            
    if (it + 1) == max_iter:
        warnings.warn('Max iteration reached, increase max_iter.')
        
    return P, Q_T

In [4]:
R = data.fillna(0).values
P, Q_T = matrix_factorization(R, K=3, lr=1e-4, max_iter=10000, tol=.5)
P, Q_T

(array([[-0.29884453,  1.95441528,  0.74581923],
        [-0.17983489,  1.21057942,  0.6202656 ],
        [ 0.69013819,  0.09251781,  1.75027554],
        [ 0.51037489, -0.05586902,  1.50002273],
        [ 2.24577792,  0.48213285,  0.17997642]]),
 array([[-0.42852303,  0.24448052,  1.84583415,  1.65561087],
        [ 2.12095506,  0.97025234,  0.50324166,  0.19773331],
        [ 0.87259485,  0.22845343, -0.4744673 ,  1.95514952]]))

In [5]:
R - P.dot(Q_T)

array([[ 0.07591323,  1.0064007 , -0.07805891, -0.34987087],
       [ 0.8141115 , -1.27230319,  0.01702713, -0.15434727],
       [-0.42776743,  0.34165258, -0.48999094,  0.41705549],
       [ 0.02829097, -0.41325501, -0.20224005,  0.23329622],
       [-0.21726104, -0.05795571,  0.69743001, -0.16534888]])

In [6]:
print('RMSE = {}'.format(np.sqrt(np.mean((R - P.dot(Q_T)) ** 2))))

RMSE = 0.4998854949737238
